The latest conversations about what to focus on in our living database construction have centered on the intersection with GAP species habitat mapping. This notebook does some additional work on building the relevant data and summarizes results.

The database for this work currently sits in a MongoDB database on our ESIP Testbed R&D space (DataDistillery). We're currently in the process of spinning this up for production, which will change the interface methods and expose everything via a public API. In the meantime, access is handled through the latest version of the [pybis package](https://github.com/usgs-bis/pybis), which can be installed via pip. The db module in that package requires a set of environment variables with connection information for the database that is only available to our team. You can enter those in a number of ways depending on how you're running Jupyter (virtual environment, etc.), including right here in the notebook using %env magic in a cell.

In [3]:
from pybis import db
from IPython.display import display
import pandas as pd
import requests

bisDB = db.Db.connect_mongodb("bis")
esaWPSpecies = bisDB["FWS_Work_Plan_Species"]

# GAP Species Habitat Maps
The Gap Analysis Project in Core Science Analytics, Synthesis and Library has produced species distribution maps for the terrestrial vertebrate species with range in CONUS. These are 30m raster products based on the GAP Land Cover data from 2001. Many of these distribution maps have been online for many years, particularly those that were started as regional products.

The following code block runs through the set of FWS Work Plan Species for which we determined a match to GAP species and hits an experimental API to pull back state-level metrics and stash those in the database for further reference. These metrics are based on an analytical database that is also undergoing review currently.

We will soon be abstracting this database call through the bis-api. What's here now is a low-level interaction with an ElasticSearch index from a staging API that is only accessible from the DOI network (including VPN), so don't count on it being fully available all the time.

In [9]:
for record in esaWPSpecies.find({"GAP":{"$exists":True}},{"Submitted Data":1,"GAP":1}):
    print (record["Submitted Data"]["Scientific Name"], "--", record["Submitted Data"]["Common Name"], "-- GAP Species Code:", record["GAP"]["gap_speciescode"])
    esResults = requests.get("https://master.staging.sciencebase.gov/bis/api/v0.1/elastic/search/bis__bis__state_species_prot_summ?q=%7B%22query%22%3A%7B%22match%22%3A%7B%22properties.sppcode%22%3A%22"+record["GAP"]["gap_speciescode"]+"%22%7D%7D%7D", verify=False).json()
    gapAnalysisByState = []
    for hit in esResults['hits']['hits']:
        gapAnalysisByState.append(hit['_source']['properties'])
    print(esaWPSpecies.update_one({'_id':record['_id']},{'$set':{'GAP Metrics':gapAnalysisByState}}))


Ambystoma barbouri -- streamside salamander -- GAP Species Code: aSTRSx


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Batrachoseps campi -- Inyo Mountains slender salamander -- GAP Species Code: aINSAx


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Batrachoseps minor -- lesser slender salamander -- GAP Species Code: aLSSAx


...

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Pseudemys rubriventris -- northern red-bellied cooter (range- wide) -- GAP Species Code: rNRBCx


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Tantilla oolitica -- Rim Rock crowned snake -- GAP Species Code: rRRCSx


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Now that we grabbed up and cached the basic metrics data, we can exploit it to build out a report of some kind. One of the things we might want to look at is any difference in the states for which the GAP models indicate possible species range and what FWS is currently considering the states in the species range. We can now look at that pretty readily with the data we've assembled thus far. This code block simply builds out lists of states provided as species range in the FWS work plan, list of the states where the GAP analysis indicates states with potentially suitable habitat, and compares the two. In cases where the GAP data indicates there may be species range in states not identified as yet in the FWS work plan, I put the GAP metrics data into a dataframe and display it for further discussion.

In [34]:
for record in esaWPSpecies.find({"GAP":{"$exists":True}},{"Submitted Data":1,"FWS Range":1,"GAP":1,"GAP Metrics":1}):
    fwsStates = [list(s.values())[0]['name'] for s in record['FWS Range']['US States']]
    gapStates = [m['state_name'] for m in record['GAP Metrics'] if m['state_name'] != 'CONUS']
    print('==== FWS Work Plan Information ====')
    display(record['Submitted Data'])
    print('==== GAP Matched Record ====')
    display(record['GAP'])
    print('==== State Range Comparison ====')
    print('FWS States', fwsStates)
    print('GAP States', gapStates)
    print('Agreement between FWS and GAP states', list(set(fwsStates) & set(gapStates)))
    print('FWS States not in GAP States', list(set(fwsStates) - set(gapStates)))
    print('GAP States not in FWS States', list(set(gapStates) - set(fwsStates)))
    
    if len(list(set(gapStates) - set(fwsStates))) > 0:
        display(pd.DataFrame(record['GAP Metrics']))
        
    print('*************************************')


==== FWS Work Plan Information ====


{'Common Name': 'streamside salamander',
 'Grouping': 'Amphibians',
 'Lead FWS Region': 'R4',
 'Scientific Name': 'Ambystoma barbouri',
 'Species Range': 'AL, KY, OH, TN, WV',
 'Species Record Reference': 'https://ecos.fws.gov/ecp/species/9776'}

==== GAP Matched Record ====


{'commonname': 'Streamside Salamander',
 'doi': 'doi:10.5066/F74J0CCV',
 'gap_speciescode': 'aSTRSx'}

==== State Range Comparison ====
FWS States ['Alabama', 'Kentucky', 'Ohio', 'Tennessee', 'West Virginia']
GAP States ['Ohio', 'West Virginia', 'Illinois', 'Tennessee', 'Indiana', 'Kentucky']
Agreement between FWS and GAP states ['Tennessee', 'Kentucky', 'Ohio', 'West Virginia']
FWS States not in GAP States ['Alabama']
GAP States not in FWS States ['Illinois', 'Indiana']


...

==== State Range Comparison ====
FWS States ['District of Columbia', 'Delaware', 'Massachusetts', 'Maryland', 'North Carolina', 'New Jersey', 'Pennsylvania', 'Virginia', 'West Virginia']
GAP States ['Pennsylvania', 'District of Columbia', 'Delaware', 'North Carolina', 'Maryland', 'Massachusetts', 'New Jersey', 'Virginia', 'West Virginia']
Agreement between FWS and GAP states ['District of Columbia', 'New Jersey', 'Virginia', 'Maryland', 'Delaware', 'Massachusetts', 'Pennsylvania', 'West Virginia', 'North Carolina']
FWS States not in GAP States []
GAP States not in FWS States []
*************************************
==== FWS Work Plan Information ====


{'Common Name': 'Rim Rock crowned snake',
 'Grouping': 'Reptiles',
 'Lead FWS Region': 'R4',
 'Scientific Name': 'Tantilla oolitica',
 'Species Range': 'FL',
 'Species Record Reference': 'https://ecos.fws.gov/ecp/species/1497'}

==== GAP Matched Record ====


{'commonname': 'Rim Rock Crowned Snake',
 'doi': 'doi:10.5066/F78914BQ',
 'gap_speciescode': 'rRRCSx'}

==== State Range Comparison ====
FWS States ['Florida']
GAP States ['Florida']
Agreement between FWS and GAP states ['Florida']
FWS States not in GAP States []
GAP States not in FWS States []
*************************************
